In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
from IPython.core.display import display, HTML
from IPython.lib.pretty import pprint
from itertools import chain, tee
from functools import partial, reduce
from dominate import tags as html
import pandas as pd

In [5]:
from mockdown.model import View
from mockdown.model.constraint import IConstraint

In [6]:
# Some utility functions.

def to_series(constraint: IConstraint) -> pd.Series:
    return pd.Series(constraint.to_dict)

def set_to_dataframe(*constraints: IConstraint):
    rows = map(lambda c: c.to_dict(), constraints)
    df = pd.DataFrame(rows)
    return df

# Gratuitous rainbow bullshit.

HTML("""
<style>
.rainbow-text {
  background-image: repeating-linear-gradient(45deg, violet, indigo, blue, green, yellow, orange, red, violet);
  text-align: center;
  background-size: 800% 800%;
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  animation: rainbow 4s ease infinite;
}

@keyframes rainbow { 
    0%{background-position:0% 50%}
    50%{background-position:100% 25%}
    100%{background-position:0% 50%}
}
</style>
""")

# Building a View

Always use the `ViewBuilder` class to construct views. This is necessary as we internally keep both `children` and `parent` links, and `View` is a frozen (immutable) dataclass. 

`ViewBuilder` performs some arcane unsafe voodoo to sidestep frozenness and recursively build a doubly-linked tree that `View.__init__` cannot sanely do. 

It also performs more aggressive validation and implements some ✨ <span class="rainbow-text">DSL MAGIC</span> ✨.

In [7]:
from mockdown.model.view import ViewBuilder as V

sample_view_1 = V("p", (0, 0, 100, 100), [
    ("a1", (10, 10, 45, 90), [
        ("b11", (20, 20, 35, 45)),
        ("b12", (20, 55, 35, 80))
    ]),
    ("a2", (55, 10, 90, 90), [
        ("b21", (65, 20, 80, 45)),
        ("b22", (65, 55, 80, 80))
    ])
]).to_view()

sample_view_2 = V("p", (0, 0, 200, 100), [
    ("a1", (10, 10, 45, 90), [
        ("b11", (20, 20, 35, 45)),
        ("b12", (20, 55, 35, 80))
    ]),
    ("a2", (55, 10, 190, 90), [
        ("b21", (65, 20, 180, 45)),
        ("b22", (65, 55, 180, 80))
    ])
]).to_view()

sample_views = [sample_view_1, sample_view_2]

# Displaying Views

The `display_view` utility method can be used to display a visual representation of a view hierarchy. 

This method optionally takes `visible_pairs` and `constraints` keyword arguments. If either of these are provided, they will be overlayed. 

In [8]:
from mockdown.display.view import display_view

disp_view = partial(display_view, 
                    scale=3, 
                    extra_styles=("margin: 1rem;"
                                  "display: inline-block;"))
container = html.div(*map(disp_view, sample_views))
display(HTML(str(container)))

In [7]:
from mockdown.visibility import visible_pairs

edge_pairs = list(map(partial(visible_pairs, deep=True), sample_views))
anchor_pairs = list(map(lambda pairs: [(e1.anchor, e2.anchor) for (e1, e2) in pairs], edge_pairs))

In [8]:
container = html.div()

for i in range(len(sample_views)):
    view_div = display_view(sample_views[i], 
                            visible_pairs=edge_pairs[i], 
                            scale=3,
                            extra_styles=("margin: 1rem;"
                                          "display: inline-block;"))
    container.add(view_div)

html_str = str(container)
display(HTML(html_str))

In [9]:
from mockdown.logic import valid_constraints

constraint_sets = list()

for i in range(len(sample_views)):
    constraint_sets += [list(valid_constraints(sample_views[i], anchor_pairs[i]))]

for constraint_set in constraint_sets:
    display(pd.DataFrame(map(IConstraint.to_dict, constraint_set[0:10])))

,y,op,a,x,b,sample_count,priority
0,a2.left,≤,1.0,a1.right,0,0,1000
1,a2.left,≥,1.0,a1.right,0,0,1000
2,b12.top,≤,1.0,b11.bottom,0,0,1000
3,b12.top,≥,1.0,b11.bottom,0,0,1000
4,b22.top,≤,1.0,b21.bottom,0,0,1000
5,b22.top,≥,1.0,b21.bottom,0,0,1000
6,a1.top,≤,1.0,p.top,0,0,1000
7,a1.top,≥,1.0,p.top,0,0,1000
8,a1.left,≤,1.0,p.left,0,0,1000
9,a1.left,≥,1.0,p.left,0,0,1000


,y,op,a,x,b,sample_count,priority
0,a2.left,≤,1.0,a1.right,0,0,1000
1,a2.left,≥,1.0,a1.right,0,0,1000
2,b12.top,≤,1.0,b11.bottom,0,0,1000
3,b12.top,≥,1.0,b11.bottom,0,0,1000
4,b22.top,≤,1.0,b21.bottom,0,0,1000
5,b22.top,≥,1.0,b21.bottom,0,0,1000
6,a1.top,≤,1.0,p.top,0,0,1000
7,a1.top,≥,1.0,p.top,0,0,1000
8,a1.left,≤,1.0,p.left,0,0,1000
9,a1.left,≥,1.0,p.left,0,0,1000


In [10]:
container = html.div()

for i in range(len(sample_views)):
    view_div = display_view(sample_views[i], 
                            constraints=constraint_sets[i], 
                            scale=3,
                            extra_styles=("margin: 1rem;"
                                          "display: inline-block;"))
    container.add(view_div)

html_str = str(container)
display(HTML(html_str))

In [11]:
from mockdown.model.constraint import IConstraint

all_constraints = list(set().union(*constraint_sets))

trained_constraints = [constraint.train_view_many(*sample_views) for constraint in all_constraints]

# for constraint in trained_constraints:
#     print(str(constraint))
# print("...")

pd.DataFrame(map(IConstraint.to_dict, trained_constraints))

,y,op,a,x,b,sample_count,priority
0,b12.right,≤,1.0,a1.right,-10,2,1000
1,b22.left,≤,1.0,a2.left,10,2,1000
2,a2.bottom,≥,1.0,p.bottom,-10,2,1000
3,b22.bottom,≤,1.0,a2.bottom,-10,2,1000
4,a2.left,≤,1.0,a1.right,10,2,1000
5,b12.right,≤,1.0,b11.right,0,2,1000
6,b12.top,≤,1.0,b11.bottom,10,2,1000
7,b21.right,≥,1.0,b22.right,0,2,1000
8,a1.left,≤,1.0,p.left,10,2,1000
9,a1.bottom,≤,1.0,p.bottom,-10,2,1000
